# October 27th 2022 update

This is a modification of the script that Julia originally wrote to summarize the GORG classifier outputs into read counts per gene per taxonomy. After extensive examination of the GORG classifier results and reading the Kaiju paper I decided that I can just use the "taxonomy_ids" which corresponds to the "taxonomy_lineage" column as my groupby column. The "taxonomy_id" is the highest classification with best matches. eg if it matches two species that are part of the same genus the taxonomy_ids is the genus those species belong to.

In order to keep all hypothetical genes from being combined into a single "hypothetical protein" and also to ensure that slight cell to cell variations of the same gene were clustered the same I ran mmseqs on the same file that was used to generate the GORG database to cluster the proteins together. Then I used the gene_cluster_rep as the unique identifier for a cluster of genes. I used the first gene in the GORG "sequence_ids" file to identify the gene_cluster_rep sequence and then grouped the dataframe on the gene_cluster_rep column and the taxonomy_ids column. This produces a file with the number of reads assigned to each gene for each taxonomic lineage. Once I have this file I can parse it into a wide output (see the next notebook) and then proceed with differential analyses and any other analyses that I am intersted in doing.

All together this notebook takes ~2 hours to process ~1.124 billion RNA reads when run in a standard interactive job with 10Gb and 8 CPUs


An average of ~10% of reads were recruiting to the genes. I need to process the output files to see what percent of these reads are trna or other ribosomal proteins but that should be pretty straightforward.

In [1]:
import glob
from collections import defaultdict
import pandas as pd
import os.path as op
import math
import os 
import sys
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter

import sqlite3
import sqlalchemy

def safe_makedir(dname):
    """
    Make a directory if it doesn't exist, handling concurrent race conditions.
    """
    if not dname:
        return dname
    num_tries = 0
    max_tries = 5
    while not os.path.exists(dname):
        try:
            os.makedirs(dname)
        except OSError:
            if num_tries > max_tries:
                raise
            num_tries += 1
            time.sleep(2)
    return dname

# import my custom functions most of these are ease of life functions and are not necessary for a lot of my notebooks
sys.path.insert(0, '/home/jmunson-mcgee/')
from JMM_functions import *

pd.set_option("display.max_rows", 6)

outdir='/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/RNA_GORG_recruit/Summary_files'
safe_make_dir(outdir)
os.chdir(outdir)

    
matplotlib.__version__

read_files = glob.glob('/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/RNA_GORG_recruit/annotations/*.txt.gz')

Folder already exists


In [2]:
# file that indicates which sags belong to which category

cat_file = '/mnt/scgc/simon/microg2p/Melody/BLM1/Data/BLM1_Qc_MELODY_210625_VH00511_17_AAAG2WYHV_assembly_stats.csv'
namecolumnid = 'well'
categorycolumnid = 'GTDB_classification'

#catdf_file = '/mnt/scgc/simon/microg2p/analyses/ani/All_GoM_SAGs_1cell_20kb_decon_clusters_added.csv'
catdf = pd.read_csv(cat_file)


gadict = {l[namecolumnid]:l[categorycolumnid] for i, l in catdf.iterrows()}


## NOTE
The mmseqs clustering has to be run on exactly the same file that was used to create the classifier database. If Greg/Ramunas want to continue to update the classifier and make it more user friendly then this is probably something that should be included before producing the output summary file.

In [3]:
mmseqs_cluster_file = '/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/mmseqs2/BLM1_GORG_genes_60minid_m80.tsv'

head = ['gene_cluster_rep', 'gene']
gcdf = pd.read_csv(mmseqs_cluster_file,
                         sep='\t', names=head)
# this gets rid of the tax_id data that is at the end of the gene_name and gene_cluster_rep strings. 
gcdf['gene_cluster_rep'] = ["_".join(i.split("_")[:-1]) for i in gcdf['gene_cluster_rep']]
gcdf['gene'] = ["_".join(i.split("_")[:-1]) for i in gcdf['gene']]

# for some reason the '-' were removed in the creation of the GORG database this time so I have to modify things to include this character
gcdf['gene_cluster_rep']=gcdf['gene_cluster_rep'].str[:2] + '-' + gcdf['gene_cluster_rep'].str[2:5] + '-'+gcdf['gene_cluster_rep'].str[5:]
gcdf['gene']=gcdf['gene'].str[:2] + '-' + gcdf['gene'].str[2:5] + '-'+gcdf['gene'].str[5:]

gcdf

,gene_cluster_rep,gene
0,AM-294-P06_N42;11716;12369,AM-294-P06_N42;11716;12369
1,AM-294-P06_N42;11716;12369,AM-307-N22_N5;46105;46758
2,AM-294-E20_N5;7770;9170,AM-294-E20_N5;7770;9170
...,...,...
124814,AM-294-L14_N3;8038;8610,AM-294-E11_N9;13611;14183
124815,AM-302-A21_N4;6358;7056,AM-302-A21_N4;6358;7056
124816,AM-302-A21_N4;6358;7056,AM-304-G09_N3;29860;30558


In [4]:
# you should be able to get rid of this cell it is left over from a different way to identify if 
# reads were shared between tax lineages but the GORG tax lineage should take care of this
gcdf['gene_sag'] = [i.split("_")[0] for i in gcdf['gene']]
gcdf['gene_genus'] = [gadict[i] for i in gcdf['gene_sag']]

# this counts the number of genes represented in each gene cluster
genus_per_cluster = gcdf[gcdf['gene_genus'] != 'Unclassified'].drop_duplicates(subset=['gene_cluster_rep','gene_genus']).groupby('gene_cluster_rep', as_index=False)['gene'].count().rename(columns={'gene':'genus_count'})

genus_per_cluster['gene_cluster_genus_status'] = ['exclusive' if i == 1 else 'shared' for i in genus_per_cluster['genus_count']]
gcdf2 = gcdf.merge(genus_per_cluster[['gene_cluster_rep','gene_cluster_genus_status']], how='left')

gcdf2['gene_cluster_genus_status'] = gcdf2['gene_cluster_genus_status'].fillna('unclassified_only')



Counter(gcdf2['gene_cluster_genus_status']).most_common()

[('shared', 69140), ('exclusive', 55677)]

### Notes from 2022-10-26
The next two cells are code that Jacob started writing  on Monday October 24th 2022 to parse the results. I spent a lot of time going through the Kaiju paper as well as the GORG output files and I have some thoughts. The taxid that is assigned in the 'taxonomy_id' and the 'taxonomic_lineage' are the highest taxa (eg the lca). If one cell has a higher classification ranking it is put in the 'taxonomy_ids_lca' oclumn. As a result of this all that I need to do is add the 'gene_cluster_rep' and then group the reads by the 'taxonomy_id' and 'gene_cluster_rep'.

In [5]:
# I can just pick one of the output files for the header because they are all identical. This is because I had to chunk the files.
names=pd.read_csv('/mnt/scgc/simon/microg2p/JdF_analysis/analysis/JdF_GORG_RNA_recruit/annotations/All_10thiosulfate_reads_noMTST_annotated.txt.gz',
           compression='gzip', nrows=0, sep='\t')
header=names.columns.to_list()

header

['status',
 'sequence_id',
 'taxonomy_id',
 'length',
 'taxonomy_ids_lca',
 'sequence_ids_lca',
 'protein_sequence',
 'taxonomic_lineage',
 'prokka_EC_number',
 'prokka_product',
 'swissprot_gene',
 'swissprot_EC_number',
 'swissprot_eggNOG',
 'swissprot_KO',
 'swissprot_Pfam',
 'swissprot_CAZy',
 'swissprot_TIGRFAMs']

In [6]:
gcdf2

,gene_cluster_rep,gene,gene_sag,gene_genus,gene_cluster_genus_status
0,AM-294-P06_N42;11716;12369,AM-294-P06_N42;11716;12369,AM-294-P06,d__Bacteria;p__Desulfobacterota;c__BSN033;o__B...,exclusive
1,AM-294-P06_N42;11716;12369,AM-307-N22_N5;46105;46758,AM-307-N22,d__Bacteria;p__Desulfobacterota;c__BSN033;o__B...,exclusive
2,AM-294-E20_N5;7770;9170,AM-294-E20_N5;7770;9170,AM-294-E20,d__Bacteria;p__Firmicutes_B;c__Desulfotomaculi...,exclusive
...,...,...,...,...,...
124814,AM-294-L14_N3;8038;8610,AM-294-E11_N9;13611;14183,AM-294-E11,d__Bacteria;p__Firmicutes_B;c__Desulfotomaculi...,exclusive
124815,AM-302-A21_N4;6358;7056,AM-302-A21_N4;6358;7056,AM-302-A21,d__Archaea;p__Thermoproteota;c__Bathyarchaeia;...,exclusive
124816,AM-302-A21_N4;6358;7056,AM-304-G09_N3;29860;30558,AM-304-G09,d__Archaea;p__Thermoproteota;c__Bathyarchaeia;...,exclusive


In [22]:
import time
start_time = time.time()

# specify columns that need to have whitespaces stripped
cols = ['taxonomic_lineage', 'gene_cluster_rep']

for file in read_files:
    print(file)
    
    # extract the filename and modify it to use as the output file
    base=os.path.basename(file)
    base=base[:-7]
    out_name=(base+'_RNA_GORG_results.csv')
    
    
    chunksize = 100000
    counter=1
    check=1
    # read the file in chunks and group the reads by each gene/tax lineage for each chunk
    for chunk in pd.read_csv(file, compression='gzip', sep='\t', chunksize=chunksize, skiprows=0, names=header):
        # tracker to keep track of the progress
        if counter % 10 == 0:
            print('the number of processed reads is',counter*chunksize, 'the total number of classified reads is', DF_summary.number_of_reads.sum(), DF_summary.number_of_reads.sum()/(counter*chunksize)*100, '% of reads recruiting')
            
        # read in the first chunk process it and store it as a summary df
        if check==1:
            # extract classified reads only
            DF_summary = chunk[(chunk['status'] == 'C')]
            if len(DF_summary)==0:
                # for some of the samples there are no classified reads this check makes sure that this first part is repeated
                # until there is at least one classified read and then it moves on to the second part that stacks the chunks
                check=1
                continue
                
            else:
                check=2
                
                # extract the first gene that the read is assigned to
                DF_summary['sequence_ids_lca']=[i.split(",")[0] if "," in i else i for i in DF_summary['sequence_ids_lca']]

                # for these samples only I have to modify the gene name to add the '-' back in
                DF_summary['sequence_ids_lca']=DF_summary['sequence_ids_lca'].str[:2] + '-' + DF_summary['sequence_ids_lca'].str[2:5] + '-'+DF_summary['sequence_ids_lca'].str[5:]
            
                # merge the file with the gcdf which has the gene_cluster_rep (a unique identifier) for every gene
                DF_summary=DF_summary.merge(gcdf2, left_on='sequence_ids_lca', right_on='gene', how='left')

                # extract the gene annotation data from the summary file and drop duplicates
                genes_summary=DF_summary[['gene_cluster_rep', 'prokka_EC_number', 'prokka_product', 'swissprot_gene',
                                          'swissprot_EC_number', 'swissprot_eggNOG', 'swissprot_KO', 'swissprot_Pfam',
                                          'swissprot_CAZy', 'swissprot_TIGRFAMs']].copy()
                genes_summary.drop_duplicates(inplace=True)
                
                
                # group the df by taxonomic lineage, and gene cluster
                DF_summary=DF_summary.groupby(['taxonomic_lineage', 'gene_cluster_rep'], as_index=False, dropna=False)['status'].count()
                DF_summary.rename(columns={'status':'number_of_reads'}, inplace=True)
                                
                
        # read all other chunks into a temporary df and process them
        else:
            # Same as above but store each successive chunk as a tempory_df
            tdf_summary = chunk[(chunk['status'] == 'C')]
            if len(tdf_summary)==0:
                # proceed to the next chunk if there are no classified reads
                continue
            
            else:               
                
                tdf_summary['sequence_ids_lca']=[i.split(",")[0] if "," in i else i for i in tdf_summary['sequence_ids_lca']]
                # for these samples only I have to modify the gene name to add the '-' back in
                tdf_summary['sequence_ids_lca']=tdf_summary['sequence_ids_lca'].str[:2] + '-' + tdf_summary['sequence_ids_lca'].str[2:5] + '-'+tdf_summary['sequence_ids_lca'].str[5:]
            
                tdf_summary=tdf_summary.merge(gcdf2, left_on='sequence_ids_lca', right_on='gene', how='left')
        
                # extract all the gene information from this chuink and drop duplicates
                tgenes_summary=tdf_summary[['gene_cluster_rep', 'prokka_EC_number', 'prokka_product', 'swissprot_gene',
                                          'swissprot_EC_number', 'swissprot_eggNOG', 'swissprot_KO', 'swissprot_Pfam', 
                                          'swissprot_CAZy', 'swissprot_TIGRFAMs']].copy()
                tgenes_summary.drop_duplicates(inplace=True)
                
                
                tdf_summary=tdf_summary.groupby(['taxonomic_lineage', 'gene_cluster_rep'], as_index=False, dropna=False)['status'].count()
                tdf_summary.rename(columns={'status':'number_of_reads'}, inplace=True)
               
                
                # merge the temporary df with the summary df and convert nan to 0 (this allows for adding togehter thecolumns)
                DF_summary=DF_summary.merge(tdf_summary, on=['taxonomic_lineage', 'gene_cluster_rep'], how='outer')
                DF_summary["number_of_reads_x"].fillna(0, inplace=True)
                DF_summary["number_of_reads_y"].fillna(0, inplace=True)
        
                # add the read numbers, and drop the read number columns that were duplicated upon the merge
                DF_summary['number_of_reads']=DF_summary['number_of_reads_x']+DF_summary['number_of_reads_y']
                DF_summary.drop(columns=['number_of_reads_x', 'number_of_reads_y'], inplace=True)
                
                
                
                # stack gene data with initial gene data and drop duplicates This will be appended at the end
                genes_summary=pd.concat([genes_summary, tgenes_summary], ignore_index=True, axis=0)
                genes_summary.drop_duplicates(inplace=True)
                

        counter+=1
        
    # add the gene data
    if check==2:
        
        genes=genes_summary['gene_cluster_rep'].unique()
        final_genes_summary=pd.DataFrame()
        for i in genes:
            tdf_genes=genes_summary[genes_summary['gene_cluster_rep']==i]
            # if there is only one product use that
            if len(tdf_genes['prokka_product'].unique())==1:
                final_genes_summary=pd.concat([final_genes_summary, tdf_genes])
            # if there are more than one product get rid of the hypothetical
            else:
                tdf_genes=tdf_genes[tdf_genes['prokka_product']!='hypothetical protein']
                if len(tdf_genes['prokka_product'].unique())>1:
                    # if there are still multiple products use only the first one
                    tdf_genes=tdf_genes.iloc[0]
                    final_genes_summary=pd.concat([final_genes_summary, tdf_genes])
                    
                else:
                    final_genes_summary=pd.concat([final_genes_summary, tdf_genes])
                
        
        DF_summary=DF_summary.merge(final_genes_summary, on='gene_cluster_rep', how='left')
        DF_summary.to_csv(os.path.join(outdir, out_name))
    
print('DONE')

print("It took --- %s seconds --- to process all of the RNA reads" % (time.time() - start_time))

/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/RNA_GORG_recruit/annotations/cDNA4-A01_joined_annotated.txt.gz


/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

the number of processed reads is 1000000 the total number of classified reads is 4660.0 0.466 % of reads recruiting
the number of processed reads is 2000000 the total number of classified reads is 9713.0 0.48564999999999997 % of reads recruiting
the number of processed reads is 3000000 the total number of classified reads is 14990.0 0.49966666666666665 % of reads recruiting
the number of processed reads is 4000000 the total number of classified reads is 20171.0 0.504275 % of reads recruiting
the number of processed reads is 5000000 the total number of classified reads is 25384.0 0.50768 % of reads recruiting
the number of processed reads is 6000000 the total number of classified reads is 30690.0 0.5115 % of reads recruiting
the number of processed reads is 7000000 the total number of classified reads is 35870.0 0.5124285714285715 % of reads recruiting
the number of processed reads is 8000000 the total number of classified reads is 41069.0 0.5133625 % of reads recruiting
the number of p

### The noncompetitive version below was very specific for Melodies project where she wanted to identify all cells that a read recruited to

In [23]:


# Redo this in a noncompetitive version so that there is a document that has all the reads to genes in SAGs.

start_time = time.time()
for file in read_files:
    print(file)
    
    # extract the filename and modify it to use as the output file
    base=os.path.basename(file)
    base=base[:-7]
    out_name=(base+'_noncompetitive_RNA_GORG_results.csv')
    
    
    chunksize = 100000
    counter=1
    check=1
    # read the file in chunks and group the reads by each gene/tax lineage for each chunk
    for chunk in pd.read_csv(file, compression='gzip', sep='\t', chunksize=chunksize, skiprows=0, names=header):
        # tracker to keep track of the progress
        if counter % 10 == 0:
            print('the number of processed reads is',counter*chunksize)
            
        # read in the first chunk process it and store it as a summary df
        if check==1:
            # extract classified reads only
            DF_summary = chunk[(chunk['status'] == 'C')]
            if len(DF_summary)==0:
                check=1
                continue
                
            else:
                check=2


  # copy the df into a new df that I can use to generate noncompetitive counts of all the SAGs that a read was assigned to
                DF_explode=DF_summary.copy()

                DF_explode=DF_explode.set_index(['status', 'sequence_id', 'length', 'taxonomy_id', 'taxonomy_ids_lca', 'protein_sequence', 'taxonomic_lineage', 'prokka_EC_number', 'prokka_product', 'swissprot_gene', 'swissprot_EC_number', 'swissprot_eggNOG', 'swissprot_KO', 'swissprot_Pfam', 'swissprot_CAZy', 'swissprot_TIGRFAMs']).apply(lambda x: x.str.split(',').explode()).reset_index()
                DF_explode['SAG']=DF_explode['sequence_ids_lca'].str[:2] + '-' + DF_explode['sequence_ids_lca'].str[2:5] + '-'+DF_explode['sequence_ids_lca'].str[5:8]
                # for some reason this adds an extra row for each read the is recruited so I am getting rid of that.
                DF_explode=DF_explode[DF_explode['SAG']!='--']
                
                # extract gene annotations for per SAG read counts
                DF_explode_gene_annotations=DF_explode[['prokka_EC_number', 'prokka_product', 'swissprot_gene', 'swissprot_EC_number', 'swissprot_eggNOG', 'swissprot_KO', 'swissprot_Pfam', 'swissprot_CAZy', 'swissprot_TIGRFAMs', 'sequence_ids_lca']].copy()
                
                DF_explode_summary=DF_explode.groupby(['SAG', 'sequence_ids_lca'], as_index=False, dropna=False)['status'].count()
                DF_explode_summary.rename(columns={'status':'number_of_reads'}, inplace=True)


                
 # read all other chunks into a temporary df and 
        else:
            # Same as above but store each successive chunk as a tempory_df
            tdf_summary = chunk[(chunk['status'] == 'C')]
            if len(tdf_summary)==0:
                continue
            
            
            else:
                # copy the df into a new df that I can use to generate noncompetitive counts of all the SAGs that a read was assigned to
                tDF_explode=tdf_summary.copy()

                tDF_explode=tDF_explode.set_index(['status', 'sequence_id', 'length', 'taxonomy_id', 'taxonomy_ids_lca', 'protein_sequence', 'taxonomic_lineage', 'prokka_EC_number', 'prokka_product', 'swissprot_gene', 'swissprot_EC_number', 'swissprot_eggNOG', 'swissprot_KO', 'swissprot_Pfam', 'swissprot_CAZy', 'swissprot_TIGRFAMs']).apply(lambda x: x.str.split(',').explode()).reset_index()
                tDF_explode['SAG']=tDF_explode['sequence_ids_lca'].str[:2] + '-' + tDF_explode['sequence_ids_lca'].str[2:5] + '-'+tDF_explode['sequence_ids_lca'].str[5:8]
                # for some reason this adds an extra row so I am getting rid of that.
                tDF_explode=tDF_explode[tDF_explode['SAG']!='--']
                
                tDF_explode_gene_annotations=tDF_explode[['prokka_EC_number', 'prokka_product', 'swissprot_gene', 'swissprot_EC_number', 'swissprot_eggNOG', 'swissprot_KO', 'swissprot_Pfam', 'swissprot_CAZy', 'swissprot_TIGRFAMs', 'sequence_ids_lca']].copy()
                
                DF_explode_gene_annotations=pd.concat([DF_explode_gene_annotations, tDF_explode_gene_annotations], ignore_index=True, axis=0)
                DF_explode_gene_annotations.drop_duplicates(inplace=True)
                
                tDF_explode_summary=tDF_explode.groupby(['SAG', 'sequence_ids_lca'], as_index=False, dropna=False)['status'].count()
                tDF_explode_summary.rename(columns={'status':'number_of_reads'}, inplace=True)

                
                ### I need to merge the tmp explode df and sum the values and I think I might be done
                DF_explode_summary=DF_explode_summary.merge(tDF_explode_summary, on=['SAG', 'sequence_ids_lca'], how='outer')
                DF_explode_summary["number_of_reads_x"].fillna(0, inplace=True)
                DF_explode_summary["number_of_reads_y"].fillna(0, inplace=True)
        
                # add the read numbers, and drop the read number columns that were duplicated upon the merge
                DF_explode_summary['number_of_reads']=DF_explode_summary['number_of_reads_x']+DF_explode_summary['number_of_reads_y']
                DF_explode_summary.drop(columns=['number_of_reads_x', 'number_of_reads_y'], inplace=True)
        
        counter+=1
        

    if check==2:
        
        genes=DF_explode_gene_annotations['sequence_ids_lca'].unique()
        final_genes_summary=pd.DataFrame()
        for i in genes:
            tdf_genes=DF_explode_gene_annotations[DF_explode_gene_annotations['sequence_ids_lca']==i]
            if len(tdf_genes['prokka_product'].unique())==1:
                final_genes_summary=pd.concat([final_genes_summary, tdf_genes])
            # if there are more than one product get rid of the hypothetical
            else:
                tdf_genes=tdf_genes[tdf_genes['prokka_product']!='hypothetical protein']
                if len(tdf_genes['prokka_product'].unique())>1:
                    # if there are still multiple products use only the first one
                    tdf_genes=tdf_genes.iloc[0]
                    final_genes_summary=pd.concat([final_genes_summary, tdf_genes])
                    
                else:
                    final_genes_summary=pd.concat([final_genes_summary, tdf_genes])
                
                
        DF_explode_summary=DF_explode_summary.merge(final_genes_summary, on='sequence_ids_lca', how='left')
        DF_explode_summary.to_csv(os.path.join(outdir, out_name))


DF_explode_summary

/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/RNA_GORG_recruit/annotations/cDNA4-A01_joined_annotated.txt.gz
the number of processed reads is 1000000
the number of processed reads is 2000000
the number of processed reads is 3000000
the number of processed reads is 4000000
the number of processed reads is 5000000
the number of processed reads is 6000000
the number of processed reads is 7000000
the number of processed reads is 8000000
the number of processed reads is 9000000
the number of processed reads is 10000000
the number of processed reads is 11000000
the number of processed reads is 12000000
the number of processed reads is 13000000
the number of processed reads is 14000000
the number of processed reads is 15000000
the number of processed reads is 16000000
/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/RNA_GORG_recruit/annotations/cDNA6-A01_joined_annotated.txt.gz
/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/RNA_GORG_recruit/annotations/cDNA9-A01_joined_annotated.txt.gz
/mnt/scgc/

,SAG,sequence_ids_lca,number_of_reads,prokka_EC_number,prokka_product,swissprot_gene,swissprot_EC_number,swissprot_eggNOG,swissprot_KO,swissprot_Pfam,swissprot_CAZy,swissprot_TIGRFAMs
0,AM-294-B10,AM294B10_N17;3470;4603,18607.0,6.3.5.5,Carbamoyl-phosphate synthase small chain,NaN,NaN,"ENOG4105C1M,CCOG0505",K01956,"PF00988,CPF00117",NaN,TIGR01368
1,AM-294-D05,AM294D05_N11;8880;10043,18611.0,6.3.5.5,Carbamoyl-phosphate synthase small chain,NaN,NaN,"ENOG4105C1M,CCOG0505",K01956,"PF00988,CPF00117",NaN,TIGR01368
2,AM-294-L10,AM294L10_N20;891;1535,7457.0,NaN,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
19,AM-294-G17,AM294G17_N4;50156;50374,1.0,NaN,Putative nitrogen fixation protein YutI,NaN,NaN,"ENOG41083MR,CCOG0694",NaN,PF01106,NaN,NaN
20,AM-294-J06,AM294J06_N23;5746;6825,1.0,6.3.5.5,Carbamoyl-phosphate synthase pyrimidine-specif...,NaN,NaN,NaN,NaN,"PF00988,CPF00117",NaN,TIGR01368
21,AM-296-B22,AM296B22_N14;2166;2447,1.0,NaN,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN
